## Install and Import Dependencies

In [1]:
pip install tensorflow scikit-learn numpy

In [2]:
import numpy as np

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Resizing, Input
from tensorflow.keras.datasets import cifar10

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Load Dataset

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
X_train.shape

(50000, 32, 32, 3)

#### Preprocess Data

In [5]:
X_train = X_train / 255
X_test = X_test / 255

In [6]:
y_train = y_train.flatten()
y_test = y_test.flatten()

## Pretrained ResNet50 for Feature Extraction

#### Load Pretrained Resnet50

In [7]:
base_resnet = ResNet50(include_top = False,
                       weights = 'imagenet',
                       input_shape = (32, 32, 3))

base_resnet.trainable = False

#### Model Architecture

In [8]:
input = Input(shape = (32, 32, 3))

x = base_resnet(input)
x = GlobalAveragePooling2D()(x)

feature_extractor = Model(inputs = input, outputs = x)

#### Shape of output of feature extractor model

In [9]:
feature_extractor(X_train[1][np.newaxis, ...]).shape

TensorShape([1, 2048])

In [10]:
features = []
labels = []

#### Extracting Features (Predicting X_train, X_test so that they will have 2048 dimentions)

In [11]:
print("Extracting training features...")
train_features = feature_extractor.predict(X_train, batch_size=64, verbose=1)
print("Extracting test features...")
test_features = feature_extractor.predict(X_test, batch_size=64, verbose=1)

Extracting training features...
782/782 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step
Extracting test features...
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


## Simple Classifier for Classification Task

#### Logistic Regression

In [12]:
classifier = LogisticRegression(max_iter = 1000, multi_class = 'multinomial')

In [13]:
classifier.fit(train_features, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [14]:
y_pred_log = classifier.predict(test_features)

In [15]:
print('Accuracy of Logistic Regression :', accuracy_score(y_test, y_pred_log))

Accuracy of Logistic Regression : 0.4872


#### Random Forest

In [16]:
randomforest = RandomForestClassifier(n_estimators = 100)

In [17]:
randomforest.fit(train_features, y_train)

RandomForestClassifier()

In [18]:
y_pred_forest = randomforest.predict(test_features)

In [19]:
print('Accuracy of Random Forest :', accuracy_score(y_test, y_pred_forest))

Accuracy of Random Forest : 0.4055
